In [ ]:
from main import gps_preprocessor
from shapely.geometry import Point, LineString
import pandas as pd
import geopandas as gpd

### Set Depot Points, Boundary and Buffer here

In [ ]:
p1 = Point(19.421231015290946, -99.12768416729142)
p2 = Point(19.420298241433308, -99.12729008610742)
p3 = Point(19.419116629726144, -99.12747041774)
p4 = Point(19.41926477082061, -99.12969297748691)
p5 = Point(19.42117868427646, -99.12944573801215)

points = [p1,p2,p3,p4,p5,p1]
depot, centroid_point = gps_preprocessor.set_depot_boundary(points,buffer = 0)

### Read and Validate GPS Data here

In [ ]:
'''
error_factor here is used as 1.5, i.e., 
    relative_error = abs(velocity_gps-velocity_vehicle) / velocity_vehicle 
    =>  ( 5 - 1 ) / 1 > 1.5
    => Calculated GPS speed is way higher than observed speed of the vehicle, i.e. gps walking
    => higher error_factor will allow more GPS walking
'''
error_factor = 1.15

In [ ]:
cols = ['UnityLicensePlate', 'RowReferenceTime', 'lat', 'lon','Speed','EngineStatus']
date_col = ['RowReferenceTime']
d_format = 'ISO8601'
# D:\Extra-Projects\MGL-India\Moving Dots
file = 'D:/Extra-Projects/MGL-India/Moving Dots/trip_data_10Units_1Year_V2.csv'
#gps_preprocessor.export_speed_flag_df(file,cols,date_col,d_format,depot,error_factor)

In [ ]:
df = gps_preprocessor.read_data(file,cols,date_col,d_format)

In [ ]:
df = gps_preprocessor.validate_mandatory_cols(df,'RowReferenceTime','lat','lon','Speed','UnityLicensePlate')

In [ ]:
df = gps_preprocessor.add_date_time_month_df(df)

In [ ]:
df = df.groupby(by=['vehicleid','date']).apply(gps_preprocessor.add_prev_latlong).reset_index(drop=True)

In [ ]:
df = gps_preprocessor.check_veh_within_depot(df,depot)

In [ ]:
df = gps_preprocessor.calculate_speed_flag(df, error_factor)

In [ ]:
count_df = gps_preprocessor.depot_locator(df)

In [ ]:
df = gps_preprocessor.filterby_speed(df)

In [ ]:
df = gps_preprocessor.blank_filter(df)

In [ ]:
df = gps_preprocessor.add_dh_trips(df,centroid_point)

In [ ]:
tolerance = 0.05
def simplfy_trip_byvehicle(df, tolerance):
        gdf = gps_preprocessor.df_to_gdf(df)
        gdf_grouped = gdf.groupby(by=['vehicleid'])
        line_gdf = []
        for vehicleid, group in gdf_grouped:
            line = LineString(zip(group['lon'], group['lat']))
            simplified_line = line.simplify(tolerance)
            line_gdf.append(gpd.GeoDataFrame({'vehicleid': [group.iloc[0]['vehicleid']], 'geometry': [simplified_line]}, crs='EPSG:4326'))
        return gpd.GeoDataFrame(pd.concat(line_gdf, ignore_index=True),geometry='geometry', crs='EPSG:4326')

linestring_gdf = simplfy_trip_byvehicle(df, tolerance)

In [21]:
for _,item in linestring_gdf.iterrows():
    if item['vehicleid'] == 'VIG21426':
        print(item['geometry'].svg(scale_factor=1.0, stroke_color=None, opacity=None))

<polyline fill="none" stroke="#66cc99" stroke-width="2.0" points="-99.12829,19.42084 -99.12816,19.42081 -99.12814,19.42078 -99.12809,19.42079 -99.12823,19.42071 -99.12816,19.42071 -99.12822,19.42092 -99.12821,19.4207 -99.12809,19.42047 -99.1281,19.42065 -99.12819,19.42075 -99.12807,19.42063 -99.12796,19.42063 -99.12812,19.42083 -99.12779,19.4203 -99.12837,19.42056 -99.12815,19.42087 -99.12813,19.42057 -99.12801,19.4209 -99.12807,19.42095 -99.12798,19.42093 -99.12833,19.42053 -99.12826,19.4207 -99.12829,19.42068 -99.12837,19.4204 -99.1282,19.42069 -99.12837,19.42086 -99.12822,19.42063 -99.12818,19.42065 -99.1282,19.42066 -99.12823,19.42066 -99.12827,19.42065 -99.1283,19.4206 -99.1283,19.42031 -99.12838,19.42027 -99.12838,19.42027 -99.12837,19.42028 -99.12833,19.42035 -99.12821,19.42031 -99.12816,19.4203 -99.12817,19.42031 -99.12817,19.42031 -99.12818,19.42032 -99.12816,19.42032 -99.12816,19.42032 -99.12815,19.42033 -99.1281,19.42039 -99.1281,19.42039 -99.1284,19.42073 -99.12841,19.42072